# How to Use

1. Run everything in the **Setup** section. 
    - Make sure to change the working directory to **your** working directory. The code for this is already there.
    - Make sure the Excel document for logging the scores also exists in your working directory, and that the file name is correct.

2. Determine *if the test needs to be run* by having a good understanding of what each test is doing. 
    - Please refer to this document [here](https://086gc.sharepoint.com/:x:/r/sites/PacificSalmonTeam/Shared%20Documents/General/02%20-%20PSSI%20Secretariat%20Teams/04%20-%20Strategic%20Salmon%20Data%20Policy%20and%20Analytics/02%20-%20Data%20Governance/00%20-%20Projects/10%20-%20Data%20Quality/Presentation/DQP%20Demo.xlsx?d=wc15abe6743954df980a05f09fe99a560&csf=1&web=1&e=CJeb6h)

3. Some requirements for the datasets:
    - The data must be on the **first sheet** in the Excel document.
    - The **first row** must be the column names. 
    - The test won't run if the Excel file is open

4. After running all the tests, the Excel document for logging the scores can be uploaded to Sharepoint using the function "Saving the file to sharepoint". 

Note: The Output Reports are used for when a data steward is asking about why their dataset gets a certain score. If the metric is not in Output Reports, then running the test itself will generate an output that can be put into a report.  

See the [Readme](/README.md) for more details.

# Setup

Please run everything in the set up, and double check the working directory so that the data can be read from that same directory.

All of these functions are used in the process of calculating data quality. 

In [1]:
from IPython import get_ipython

# Clear memory
get_ipython().magic('reset -sf')

C:\Users\onakd\AppData\Local\Temp\1\ipykernel_56356\1945100915.py:4: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -sf')


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os
import re
from difflib import SequenceMatcher
from datetime import datetime
import nbformat
import gc

# Import dimensions
from dimensions.consistency import Consistency
from dimensions.accuracy import Accuracy
from dimensions.completeness import Completeness
from dimensions.uniqueness import Uniqueness
from dimensions.utils import calculate_dimension_score, calculate_DQ_grade

In [3]:
gc.collect()

11

Make sure to set to the correct working directory

In [ ]:
# Change working directory to the same place where you saved the test datasets
# os.chdir('C:/Users/luos/OneDrive - DFO-MPO/Python') #change directory
os.getcwd()  # check where the directory is (and whether the change was successful or not)
LOGGING_PATH = "/metric_output_logs/" # Where detailed metric outputs are stored if dimension has return_type set to 'dataset'
GLOBAL_USER = "username" # Username too keep track of who ran tests for output logging
DATA_FILE_PATH = "dataset.xlsx" # Path to csv or xlsx dataset
DIMENSION_SCORES = [] # Stores the final score for each dimension used to calculate final grade

# Data Quality Tests

### Consistency

#### Consistency Type 1 (C1)

Calculate consistency score of a dataset

This code is best run on CSV data where the column names are in the first row. It can also accept files that are in xlsx formats but it will only take data from the first sheet if there are more than one sheet in the excel file.

Limitations: It will not check for differences in capitalization of the same word (since all the words will be changed to lower case before the similarity score is calculated)

#### Consistency Type 2 (C2)

Calculate consistency score of datasets with a reference list

The compared columns in question must be identical to the ref list, otherwise they will be penalized more harshly.

In [ ]:
column_mapping = {
    "STOCK_CU_NAME": "CU_Display",
    "STOCK_CU_INDEX": "FULL_CU_IN",
}  # the pattern for comparison is 'dataset column' : 'reference column'

# Test Consistency Calculations
# Using default thresholds and stop words for both metrics
consitancy_tests = Consistency(
    dataset_path=DATA_FILE_PATH,
    c1_column_names=["DESCR"],
    c2_column_mapping=column_mapping,
    ref_dataset_path="Pacific Salmon Population Unit Crosswalk_Final_20240513.xlsx",
    return_type='dataset', # Can set to score or dataset (if dataset will create 1 sentence summary in output report)
    c2_stop_words=['']
    # logging_path=LOGGING_PATH # By default this is stored in memory (if set and return_type is dataset, will store csv of all numbers used in metric calculations)
)

consistancy_score = calculate_dimension_score("Consistency", scores=consitancy_tests.run_metrics(["C2"]), weights={})
DIMENSION_SCORES.append(consistancy_score)
print(consistancy_score)


Number of weights does not match number of metrics run, using default weights instead!
Weights do not add up to 1.0, using default weights instead!
{'dimension': 'Consistency', 'score': 0.993045537439778}


c:\Users\onakd\Documents\Data Quality Tests\DataQuality\dimensions\utils.py:361: DtypeWarning: Columns (59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(logging_path)


### Accuracy

#### Accuracy Type 1 (A1, Mixed Data Types, Symbols in Numerics) 

Test whether there are symbols in numerics

#### Accuracy Type 2 (A2 Outliers)

Find outliers that are 1.5 (or any threshold) times away from the inter-quartile range.

In [ ]:
# Test Accuracy Calculations
# Using default threshold, group by, and min score for A2 metric 
accuracy_tests = Accuracy(
    dataset_path=DATA_FILE_PATH,
    # selected_columns=[" Egg Target ", " Release/ Transfer Target ", " Coded Wire Tag Target ", " Fin Clip Target ", " Thermal Mark Target ", " Parentage-based Tag Target ", " PIT Tag Target "]
    selected_columns=["Escapement_Total"],
    groupby_column=['SMU'],
    return_type='dataset' # Can set to score or dataset (if dataset will create 1 sentence summary in output report)
    # logging_path=LOGGING_PATH # By default this is stored in memory (if set and return_type is dataset, will store csv of all numbers used in metric calculations)
)

accuracy_score = calculate_dimension_score("Accuracy", scores=accuracy_tests.run_metrics(["A2"]), weights={})
DIMENSION_SCORES.append(accuracy_score)
print(accuracy_score)

test 1
Column being processed: Escapement_Total
Outliers before grouping: SMU                       
ALSEK CHINOOK SALMON  3596    False
                      3597    False
                      3598    False
                      3599    False
                      3600    False
                              ...  
YUKON RIVER CHUM      3928    False
                      3929    False
                      3930    False
                      3931    False
                      3932    False
Name: Escapement_Total, Length: 4223, dtype: bool
Outliers index: MultiIndex([('ALSEK CHINOOK SALMON', 3596),
            ('ALSEK CHINOOK SALMON', 3597),
            ('ALSEK CHINOOK SALMON', 3598),
            ('ALSEK CHINOOK SALMON', 3599),
            ('ALSEK CHINOOK SALMON', 3600),
            ('ALSEK CHINOOK SALMON', 3601),
            ('ALSEK CHINOOK SALMON', 3602),
            ('ALSEK CHINOOK SALMON', 3603),
            ('ALSEK CHINOOK SALMON', 3604),
            ('ALSEK CHINOOK SALMON', 3605

### Completeness (P)

#### Completeness Type 1 (P1)

The threshold is for removing a column that meets the threshold of the percentage of blanks.

In [ ]:
completeness_tests = Completeness(
    dataset_path=DATA_FILE_PATH,
    return_type='dataset' # Can set to score or dataset (if dataset will create 1 sentence summary in output report)
    # logging_path=LOGGING_PATH # By default this is stored in memory (if set and return_type is dataset, will store csv of all numbers used in metric calculations)
)

completeness_score = calculate_dimension_score("Completeness", completeness_tests.run_metrics(), weights={})
DIMENSION_SCORES.append(completeness_score)
print(completeness_score)

Number of weights does not match number of metrics run, using default weights instead!
Weights do not add up to 1.0, using default weights instead!
{'dimension': 'Completeness', 'score': 1.0}


### Uniqueness (U)

#### Uniqueness Type 1 (U1)

Find duplicated rows.

In [ ]:
uniqueness_tests = Uniqueness(
    dataset_path=DATA_FILE_PATH,
    return_type='dataset' # Can set to score or dataset (if dataset will create 1 sentence summary in output report)
    # logging_path=LOGGING_PATH # By default this is stored in memory (if set and return_type is dataset, will store csv of all numbers used in metric calculations)
)

uniqueness_score = calculate_dimension_score("Uniqueness", uniqueness_tests.run_metrics(), weights={})
DIMENSION_SCORES.append(uniqueness_score)
print(uniqueness_score)

Duplicate Rows:
Empty DataFrame
Columns: [QTY_SOLD, DESCRIPTION, Description FR]
Index: []

Duplication Score: 100.0%
When trying to create one line summary for U1, the following error occurred: '>' not supported between instances of 'str' and 'int'
Number of weights does not match number of metrics run, using default weights instead!
Weights do not add up to 1.0, using default weights instead!
{'dimension': 'Uniqueness', 'score': 1.0}


## Determine Overall Data Quality Grade

In [9]:
# Call grade calculation here
print(f'DQ grade for this dataset is: {calculate_DQ_grade(DIMENSION_SCORES)}')

DQ grade for this dataset is: C
